In [1]:
import numpy as np
import healpy as hp
from astropy.table import Table, vstack
from filter_stars import remove_stars

# ChANGES SED Downselector

This code does the same as [downselector.victoria.ipynb](downselector.victoria.ipynb) but limited to r=21.5 so as to create the wide survey.

### Helpix parameters and useful functions

In [2]:
#First we defined the basic healpix parameters we will use. 
NSIDE = 64
npix = hp.nside2npix(NSIDE)
total_degrees_in_sky = 4.*np.pi*(180./np.pi)**2
area_per_pixel = total_degrees_in_sky/(1.*npix)

In [3]:
def radec_to_sph(ra,dc):
    
    theta = (90.-dc)*np.pi/180.
    phi   = ra*np.pi/180.
    return theta, phi

In [4]:
def get_n(ra,dec,NSIDE):
    #Convert to ra/dec
    theta, phi = radec_to_sph(ra,dec)
    #set up the healpix grid.
    n = hp.ang2pix(NSIDE, theta, phi)    
    return n

### Median F-test catalog density

We need to first obtain the median F-test catalog density, as this sets the basis for constructing the final catalogs.

In [5]:
#Read the catalog. 
Fcat_full_unfiltered = Table.read("../Victoria_v1.1_SED_catalogs/Master_Catalog_F_test_AGN_full_photometry.fits")

In [ ]:
#Filter out the remaning stars as best as we can. 
#Fcat_full = remove_stars(Fcat_full_unfiltered)
Fcat_full = Fcat_full_unfiltered

In [6]:
#Now, cut the catalog to r<21.5
cond = Fcat_full['mag_auto_r']<21.5
Fcat = Fcat_full[cond]

In [7]:
#Get the healpix pixel numbers.
n_Fcat = get_n(Fcat['ra'] , Fcat['dec'] , NSIDE)
n_Ftot = n_Fcat

In [8]:
#Get the median field density
h_Ftot = np.histogram(n_Ftot,hp.nside2npix(NSIDE), range=(0,hp.nside2npix(NSIDE)-1))[0]
median_F_number = np.median(h_Ftot[h_Ftot>0])
median_F_density = median_F_number/area_per_pixel

In [9]:
print("The median density of F-test sources is {:.1f} per sq. deg.".format(median_F_density))
print("In regions that are not gaps or deeper fields, there are a total of {} sources".format(np.sum(h_Ftot[(h_Ftot>0.2*median_F_number) & (h_Ftot<3*median_F_number)])))

The median density of F-test sources is 79.8 per sq. deg.
In regions that are not gaps or deeper fields, there are a total of 1195359 sources


In [10]:
#Now, we need to define a gap density and an overdense field density to make the final catalog. To define the gap regions, we need to get the BIC densities. 
BIC_cat_full_unfiltered = Table.read("../Victoria_v1.1_SED_catalogs/Master_Catalog_BIC_AGN_nb_le_6_full_photometry.fits")

In [ ]:
#Remove the stars. 
#BIC_cat_full = remove_stars(BIC_cat_full_unfiltered)
BIC_cat_full = BIC_cat_full_unfiltered

In [ ]:
#Now, cut the catalog to r<21.5
cond = BIC_cat_full['mag_auto_r']<21.5
BIC_cat = BIC_cat_full[cond]

In [11]:
#Get the healpix numbers
n_BIC_cat  = get_n(BIC_cat['ra'] , BIC_cat['dec'] , NSIDE)
n_BIC = n_BIC_cat

In [12]:
#Add up the number of sources per healpix pixel. 
h_BIC = np.histogram(n_BIC,hp.nside2npix(NSIDE), range=(0,hp.nside2npix(NSIDE)-1))[0]

### Estimate the total number of sources

In [13]:
#Define the main survey and the gaps. Deeper fields will just have the same source density as the rest of the wide field. 
main_survey_condition = (h_Ftot>=0.2*median_F_number)
gap_condition = (h_BIC>0) & (h_Ftot<0.2*median_F_number)

In [14]:
#Now, get the total number of sources that the final catalog will have if we keep the median number in all the cells of the main F-test survey region and half of that in the gaps. 
N_FR = len(h_Ftot[main_survey_condition])*median_F_number
N_GR = len(h_Ftot[gap_condition])*median_F_number*0.5
print(N_FR, N_GR, N_FR+N_GR)

1077226.0 220430.0 1297656.0


### Select the sources

In [15]:
#This is the histogram that will hold for each healpix pixel the number of sources in the final catalog. 
h_final = np.zeros(len(h_Ftot))

In [16]:
#We will create an extra column for each table where we will make notice of whether a given source makes it or not in the final table. 
Fcat['Final_cat']     = np.zeros(len(Fcat['ra']), dtype=np.int32)
BIC_cat['Final_cat']  = np.zeros(len(BIC_cat['ra']), dtype=np.int32)

In [17]:
#This is the main loop to select the sources.
for n in range(npix):

    #If there are no sources, skip to the next step. 
    if h_BIC[n]==0:
        continue

    #Set the target number of sources. 
    targ_num = median_F_number
    if gap_condition[n]:
        targ_num = 0.5*median_F_number
    targ_num = np.int32(targ_num)

    #Make slice conditions. 
    cond_F1 = n_Fcat==n
    cond_B1 = n_BIC_cat==n

    #Get the number of F-test sources and BIC sources in each pixel. 
    F_num = len(Fcat['ra'][cond_F1])
    B_num = len(BIC_cat['ra'][cond_B1])

    #If we have enough F-test sources, just use those and keep only the ones with the lowest probability. 
    if F_num>=targ_num:
        Fp_lim = np.sort(Fcat['Fp'][cond_F1])[targ_num-1]
        Fcat['Final_cat'][(cond_F1) & (Fcat['Fp']<=Fp_lim)] = 1

    #If we have enough sources between BIC and F-test, then use all the F-test sources and the best BIC sources. 
    elif F_num + B_num > targ_num:
        Fcat['Final_cat'][cond_F1] = 1
        aux = np.sort(BIC_cat['BIC'][cond_B1])[::-1]
        BIC_lim = aux[targ_num-F_num]
        BIC_cat['Final_cat'][(cond_B1) & (BIC_cat['BIC']>=BIC_lim)] = 1

    #Finally, if not enough between the two of them, just use all sources. 
    else:
        Fcat['Final_cat'][cond_F1] = 1
        BIC_cat['Final_cat'][cond_B1] = 1

In [18]:
#Save the catalogs.
Fcat_save = Fcat[Fcat['Final_cat']==1]
Fcat_save.write("Master_Catalog_F_test_AGN_full_photometry.downselected.wide.fits")

BIC_cat_save = BIC_cat[BIC_cat['Final_cat']==1]
BIC_cat_save.write("Master_Catalog_BIC_AGN_nb_le_6_full_photometry.downselected.wide.fits")

In [19]:
#Create a combined downselected table. 
Combined_table = vstack([Fcat_save, BIC_cat_save])

#Remove the final cat column. 
Combined_table.remove_column('Final_cat')

#Add a column with how the source was selected.
Combined_table['Selection'] = ["F"]*len(Fcat_save['ra']) + ["BIC"]*len(BIC_cat_save)

#Save the sources.
Combined_table.write("Master_Catalog_full_photometry.downselected.wide.v1.1.fits")